Data base connection 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import mysql.connector


db = mysql.connector.connect(host = "localhost",
                            username = "root",
                            password = "root",
                            database = "e_com_prj01")

cur = db.cursor()

1. List all unique cities where customers are located.

In [ ]:
query = """ select distinct customer_city as "Citys" from customers """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data)
df.head()

Count the number of orders placed in 2017

In [ ]:
query = """ select count(order_purchase_timestamp) from orders
where order_purchase_timestamp =2017"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data)
df.head()

 Find the total sales per category.

In [ ]:
query = """ select upper(products.product_category) category, round(sum(payments.payment_value),2) sales from e_com_prj01.products

join e_com_prj01.delivery
on e_com_prj01.products.product_id=e_com_prj01.delivery.product_id
join e_com_prj01.payments
on e_com_prj01.payments.order_id=e_com_prj01.delivery.order_id group by category;
"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data,columns = ["Category", "Sales"])
df


4. Calculate the percentage of orders that were paid in installments.

In [ ]:
query = ''' select (count(payment_installments)/ (select count(*)  from e_com_prj01.payments) ) * 100  percentage 
from e_com_prj01.payments
where payment_installments>=1 '''
cur.execute(query)
data = cur.fetchall()
data
df=pd.DataFrame(data[0])
df.head(1)

In [ ]:
query = """ select ((sum(case when payment_installments >= 1 then 1
else 0 end))/count(*))*100 from payments
"""

cur.execute(query)

data = cur.fetchall()

"the percentage of orders that were paid in installments is", data[0][0]

5. Count the number of customers from each state. 

In [ ]:
query = ''' select seller_state,count(seller_id) from e_com_prj01.seller
group by seller_state '''

cur.execute(query)

data = cur.fetchall()
data

Intermediate Queries
1. Calculate the number of orders per month in 2018.

In [ ]:

query = ''' select monthname(order_purchase_timestamp) months, count(order_id) order_count
from e_com_prj01.orders
where year(order_purchase_timestamp) = 2018
group by months'''
cur.execute(query)
data = cur.fetchall()
# data
df=pd.DataFrame (data,columns =["months","order_count"])
# df
plt.xticks(rotation = 45)
plt.title("Count of Orders by Months is 2018")

plt.show()


In [ ]:
query = """ select monthname(order_purchase_timestamp) months, count(order_id) order_count
from orders where year(order_purchase_timestamp) = 2018
group by months
"""

cur.execute(query)

data = cur.fetchall()
df = pd.DataFrame(data, columns = ["months", "order_count"])
o = ["January", "February","March","April","May","June","July","August","September","October"]

# ax = sns.barplot(x = df["months"],y =  df["order_count"], data = df, order = o, color = "red")
plt.xticks(rotation = 45)
# ax.bar_label(ax.containers[0])
plt.title("Count of Orders by Months is 2018")

plt.show()

2. Find the average number of products per order, grouped by customer city.

In [ ]:
query= '''with as_per_count as (select orders.order_id,orders.customer_id, count(delivery.order_id) as oc from e_com_prj01.orders join e_com_prj01.delivery on
orders.order_id = delivery.order_id
group by orders.order_id,orders.customer_id) 

select customers.customer_city, round(avg(as_per_count.oc),2) average_orders
from customers join as_per_count
on customers.customer_id = as_per_count.customer_id
group by customers.customer_city order by average_orders desc;'''

cur.execute(query)
data= cur.fetchall()
df =pd.DataFrame(data)
df

3. Calculate the percentage of total revenue contributed by each product category.

In [ ]:
query =''' with as_new_tab as (select products.product_category, round(sum(delivery.price)) Actull_sell_price ,round(sum(payments.payment_value)) act_paid_prc ,
(round(sum(payments.payment_value)) - round(sum(delivery.price)))  as  ravinew  
from products join delivery 
on products.product_id = delivery.product_id 
join payments on  payments.order_id = delivery.order_id
group by product_category)

select product_category, ravinew ,(ravinew/(select sum(ravinew) from as_new_tab) *100) as percentages from as_new_tab;'''

cur.execute(query)
data =cur.fetchall()
df = pd.DataFrame(data,columns =['product','ravinew','percentage'])
df

4. Identify the correlation between product price and the number of times a product has been purchased.

In [ ]:
query='''select count(e.product_id),product_category,round(avg(price),2) from e_com_prj01.delivery d
join e_com_prj01.products e on 
d.product_id = e.product_id
group by product_category;'''
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame (data,columns= ['Count_Of_Product','Product_Category','On_Average_Price'])
df['Count_Of_Product'],df['On_Average_Price']
import numpy as np
np.corrcoef([df['Count_Of_Product'],df['On_Average_Price']]) # it will denoted as how it co related the product price and product's numver of purchased


array([[ 1.        , -0.10631514],
       [-0.10631514,  1.        ]])

5. Calculate the total revenue generated by each seller, and rank them by revenue.

In [16]:
query='''select * ,dense_rank() over(order by a.revenue desc) as 'rank' from 
(
select distinct s.seller_id ,round(sum(d.price+d.freight_value),2) as revenue ,count(d.order_id) from  e_com_prj01.seller s 
join e_com_prj01.delivery d on
s.seller_id=d.seller_id
join e_com_prj01.products p
on d.product_id = p.product_id
group by seller_id
) as a;'''

cur.execute(query)
data = cur. fetchall()
df = pd.DataFrame( data, columns= ["Seller_Id" , "Revenue", "Product_Quantity", "Ranks" ] )
df[0:6]

,Seller_Id,Revenue,Product_Quantity,Ranks
0,4869f7a5dfa277a7dca6462dcf3b52b2,249640.70,1156,1
1,7c67e1448b00f6e969d365cea6b010ab,239536.44,1364,2
2,53243585a1d6dc2643021fd1853d8905,235856.68,410,3
3,4a3ca9315b744ce9f8e9374361493884,235539.96,1987,4
4,fa1c13f2614d7b5c4749cbc52fecda94,204084.73,586,5
5,da8622b14eb17ae2831f4ac5b9dab84a,185192.32,1551,6


Advanced Queries
1. Calculate the moving average of order values for each customer over their order history.

In [ ]:
query='''select customer_id, order_purchase_timestamp,payment_value,avg(payment_value)  over(partition by customer_id order by order_purchase_timestamp
rows between 2 preceding and current row)as mov_avg from
(select c.customer_id ,o.order_purchase_timestamp, o.order_id, p.payment_value from e_com_prj01.customers as c join e_com_prj01.orders as o 
on o.customer_id= c.customer_id join e_com_prj01.payments as p
on o.order_id = p.order_id) as a;
'''
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data)
df

2. Calculate the cumulative sales per month for each year.

In [ ]:
query = '''
SELECT 
    years, 
    months, 
    total_payments, 
    ROUND(SUM(total_payments) OVER(ORDER BY years, months), 2) AS Cumulative_sells_per_year
FROM 
(
    SELECT 
        YEAR(order_purchase_timestamp) AS years, 
        MONTHNAME(order_purchase_timestamp) AS months, 
        COUNT(o.order_id) AS order_count,
        ROUND(SUM(p.payment_value), 2) AS total_payments
    FROM 
        e_com_prj01.orders AS o 
    JOIN 
        e_com_prj01.payments AS p
    ON 
        o.order_id = p.order_id
    GROUP BY 
        years, months, MONTH(order_purchase_timestamp)
    ORDER BY 
        years, MONTH(order_purchase_timestamp)
) AS a;
'''
cur.execute(query)
data = cur.fetchall()
data
# try:
#     # Execute the query
#     cur.execute(query)

#     # Fetch the data
#     data = cur.fetchall()

    # Print the results
for row in data:
    print(row)

3. Calculate the year-over-year growth rate of total sales.

In [6]:
query =''' 
with a as(select year(order_purchase_timestamp) as years ,round (sum( payment_value ),2) as payments  from e_com_prj01.orders as od
join
e_com_prj01.payments as pay
on 
od.order_id = pay.order_id
group by years order by years desc)
select years,payments, round (((payments - lag(a.payments,1)over( order by years))/lag(a.payments,1)over( order by years))*100,2) as "YOY" from  a;
'''
cur.execute(query)
data=cur.fetchall()
df = pd.DataFrame( data, columns= ["Year","Payment","YOY"])
df

,Year,Payment,YOY
0,2016,59362.34,NaN
1,2017,7249746.73,12112.7
2,2018,8699763.05,20.0
